# 2.4 – Handling Missing Data

The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous.
In particular, many interesting datasets will have some amount of data missing.
To make matters even more complicated, different data sources may indicate missing data in different ways.

In this section, we will discuss some general considerations for missing data, discuss how Pandas chooses to represent it, and demonstrate some built-in Pandas tools for handling missing data in Python.
Here and throughout the book, we'll refer to missing data in general as *null*, *NaN*, or *NA* values.

## Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame.
Generally, they revolve around one of two strategies: using a *mask* that globally indicates missing values, or choosing a *sentinel value* that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

As in most cases where no universally optimal choice exists, different languages and systems use different conventions.
For example, the R language uses reserved bit patterns within each data type as sentinel values indicating missing data, while the SciDB system uses an extra byte attached to every cell which indicates a NA state.

## Missing Data in Pandas

The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for non-floating-point data types.

Pandas could have followed R's lead in specifying bit patterns for each individual data type to indicate nullness, but this approach turns out to be rather unwieldy.
While R contains four basic data types, NumPy supports *far* more than this: for example, while R has a single integer type, NumPy supports *fourteen* basic integer types once you account for available precisions, signedness, and endianness of the encoding.
Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

NumPy does have support for masked arrays – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad."
Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

With these constraints in mind, Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point ``NaN`` value, and the Python ``None`` object.
This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest. There's also a special integer ``NA`` object.

### ``None``: Pythonic missing data

The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.
Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

This ``dtype=object`` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects.
While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
45.3 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
960 µs ± 316 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)



The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

In [4]:
vals1

array([1, None, 3, 4], dtype=object)

In [5]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

This reflects the fact that addition between an integer and ``None`` is undefined.

### ``NaN``: Missing numerical data

The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [6]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code.
You should be aware that ``NaN`` is a bit like a data virus–it infects any other object it touches.
Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

In [7]:
1 + np.nan

nan

In [8]:
0 * np.nan

nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful:

In [9]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy does provide some special aggregations that will ignore these missing values:

In [10]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

Keep in mind that ``NaN`` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

**Important!** Make sure you have understood the difference between ``None`` and ``np.nan``.

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [11]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present.
For example, if we set a value in an integer array to ``np.nan``, it will automatically be upcast to a floating-point type to accommodate the NA:

In [12]:
x = pd.Series(range(2), dtype=int)  # values are int64 type
x

0    0
1    1
dtype: int64

In [13]:
x[0] = None  # all values are updated to float64 type
x

0    NaN
1    1.0
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the ``None`` to a ``NaN`` value.

While this type of magic may feel a bit hackish compared to the more unified approach to NA values in domain-specific languages like R, the Pandas sentinel/casting approach works quite well in practice and only rarely causes issues.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

## Integer NA

In certain cases ``NaN`` may cause serious problems. For instance, if your integer column is, say, an identifier, casting to float can be problematic. Some integers cannot even be represented as floating point numbers.

Pandas can represent integer data with possibly missing values using ``dtype=pd.Int64Dtype()``. This is an extension implemented within pandas:

In [14]:
pd.Series([1, 2, np.nan, None], dtype=pd.Int64Dtype())

0       1
1       2
2    <NA>
3    <NA>
dtype: Int64

This can also be achieved using alias "Int64":

In [15]:
pd.Series([1, 2, np.nan, None], dtype="Int64")  # try without dtype="Int64" to see the difference

0       1
1       2
2    <NA>
3    <NA>
dtype: Int64

This also works with DataFrames:

In [16]:
pd.DataFrame([1, 2, np.nan, None], dtype="Int64")  # try without dtype="Int64" to see the difference

,0
0,1
1,2
2,<NA>
3,<NA>


It is a default choice for pandas integer arrays: 

In [17]:
pd.array([1, 2, np.nan, None])

<IntegerArray>
[1, 2, <NA>, <NA>]
Length: 4, dtype: Int64

**Important!** Note the capital "I" in "Int64" used to differentiate from NumPy's "int64" dtype:

In [18]:
pd.array([1, 2, np.nan], dtype=np.int64)  # gives an error

ValueError: cannot convert float NaN to integer

## Operating on Null Values

As we have seen, Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.
To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

- ``isnull()``: Generate a boolean mask indicating missing values
- ``notnull()``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

We will conclude this section with a brief exploration and demonstration of these routines.

In [19]:
arr = pd.array([1, 2.1, np.nan, None, pd.NA])
arr

<FloatingArray>
[1.0, 2.1, <NA>, <NA>, <NA>]
Length: 5, dtype: Float64

### Detecting null values
Pandas data structures have two useful methods for detecting null data: ``isnull()`` and ``notnull()``.
Either one will return a Boolean mask over the data. For example:

In [20]:
data = pd.Series([1, np.nan, 'hello', None])

In [21]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [22]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

As mentioned in [Data Indexing and Selection](L22_Data_Indexing_and_Selection.ipynb), Boolean masks can be used directly as a ``Series`` or ``DataFrame`` index:

In [23]:
data[data.notnull()]

0        1
2    hello
dtype: object

The ``isnull()`` and ``notnull()`` methods produce similar Boolean results for ``DataFrame``s.

**Your turn.** Consider the dataframe ``A`` defined below and find:
- the number of missing values in each column
- the number of missing values in each row
- the percentage of the missing values in the data (answer: 34.29%)

In [24]:
# DataFrame A
np.random.seed(2)
A = pd.DataFrame(np.round(np.random.random((7,5)),2), columns=list("ABCDE"))
A[A < 0.25] = np.nan
A

,A,B,C,D,E
0,0.44,NaN,0.55,0.44,0.42
1,0.33,NaN,0.62,0.30,0.27
2,0.62,0.53,NaN,0.51,NaN
3,0.79,0.85,0.49,0.85,NaN
4,0.51,NaN,0.43,NaN,NaN
5,0.60,NaN,NaN,NaN,0.35
6,0.47,NaN,0.64,0.48,0.51


In [25]:
# write your code here - the number of missing values in each column



In [26]:
# write your code here - the number of missing values in each row



In [27]:
# write your code here - the percentage of missing values



### Dropping null values

In addition to the masking used before, there are the convenience methods:
- ``dropna()``, which removes NA values, and
- ``fillna()``, which fills in NA values. 

For a ``Series``, the result is straightforward:

In [28]:
data.dropna()

0        1
2    hello
dtype: object

For a ``DataFrame``, there are more options.
Consider the following ``DataFrame``:

In [29]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


We cannot drop single values from a ``DataFrame``; we can only drop full rows or full columns.
Depending on the application, you might want one or the other, so ``dropna()`` gives a number of options for a ``DataFrame``.

By default, ``dropna()`` will drop *all rows* in which *any* null value is present:

In [30]:
df.dropna()  # drops rows with NaNs

,0,1,2
1,2.0,3.0,5


Alternatively, you can drop NA values along a different axis; ``axis=1`` drops all columns containing a null value:

In [31]:
df.dropna(axis='columns')  # drops columns with NaNs

,2
0,2
1,5
2,6


But this drops some good data as well; you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values.
This can be specified through the ``how`` or ``thresh`` parameters, which allow fine control of the number of nulls to allow through.

The default is ``how='any'``, such that any row or column (depending on the ``axis`` keyword) containing a null value will be dropped.
You can also specify ``how='all'``, which will only drop rows/columns that are *all* null values:

In [32]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [33]:
df.dropna(axis='columns', how='all')  # drops all-NaN columns

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


For finer-grained control, the ``thresh`` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [34]:
df.dropna(axis='rows', thresh=3)  # drops rows that have less than 3 non-NaNs 

,0,1,2,3
1,2.0,3.0,5,NaN


Here the first and last row have been dropped, because they contain only two non-null values.

**Your turn.** Consider the dataframe ``A`` again and complete the following tasks:
- drop all rows that have any missing values (answer: no rows should remain)
- drop all rows that have less than 3 non-missing values (answer: five rows should remain)
- drop all rows that have at least 2 missing values (answer: four rows should remain)

In [35]:
# write your code here



In [36]:
# write your code here



In [37]:
# write your code here



### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.
This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values.
You could do this in-place using the ``isnull()`` method as a mask, but because it is such a common operation Pandas provides the ``fillna()`` method, which returns a copy of the array with the null values replaced.

Consider the following ``Series``:

In [38]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

We can fill NA entries with a single value, such as zero:

In [39]:
data.fillna(0) # or value=0

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [40]:
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

or mean:

In [41]:
data.fillna(data.mean()) # or value=data.mean()

a    1.0
b    2.0
c    2.0
d    2.0
e    3.0
dtype: float64

We can specify a forward-fill to propagate the previous value forward:

In [42]:
# forward-fill: 1 NaN 2 NaN 3 -> 1 1 2 2 3
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [43]:
# back-fill: 1 NaN 2 NaN 3 -> 1 2 2 3 3
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

It is also possible to fill missing values using an interpolation method:

In [44]:
# the default method is "linear": it ignores the index and treats all values as equally spaced
data.interpolate()

a    1.0
b    1.5
c    2.0
d    2.5
e    3.0
dtype: float64

For ``DataFrame``s, the options are similar, but we can also specify an ``axis`` along which the fills take place:

In [45]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [46]:
# forward-fill along columns, i.e. left-to-right
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Notice that if a previous value is not available during a forward fill, the NA value remains.

**Your turn.** Consider the dataframe ``A`` again and:
- fill the missing values with the overall mean
- fill the missing values with the corresponding column mean
- fill the missing values using the forward fill method along columns

In [47]:
# write your code here



In [48]:
# write your code here



In [49]:
# write your code here



---

## Exercises

**Exercise 2.4.1** Consider the DataFrame defined below.

In [50]:
df = pd.DataFrame({
'ord_no':[70001,np.nan,70002,70004,np.nan,70005,np.nan,70010,np.nan,70003,70012,np.nan,70013],
'purch_amt':[150.5,np.nan,65.26,110.5,np.nan,2400.6,5760,1983.43,np.nan,2480.4,250.45, 75.29,3045.6],
'ord_date':['2012-10-05','2012-09-10',np.nan,'2012-08-17',np.nan,'2012-07-27','2012-09-10','2012-10-10',np.nan,'2012-10-10',np.nan,'2012-08-17','2012-04-25'],
'entry_id':[101,102,103,104,105,106,107,108,109,110,111,112,113],
'customer_id':[3002,3001,np.nan,3003,np.nan,3001,3001,3004,np.nan,3003,3002,np.nan,3001],
'salesman_id':[5002,5003,np.nan,np.nan,5002,np.nan,5001,np.nan,np.nan,5003,np.nan,5003,np.nan]})
df

,ord_no,purch_amt,ord_date,entry_id,customer_id,salesman_id
0,70001.0,150.50,2012-10-05,101,3002.0,5002.0
1,NaN,NaN,2012-09-10,102,3001.0,5003.0
2,70002.0,65.26,NaN,103,NaN,NaN
3,70004.0,110.50,2012-08-17,104,3003.0,NaN
4,NaN,NaN,NaN,105,NaN,5002.0
5,70005.0,2400.60,2012-07-27,106,3001.0,NaN
6,NaN,5760.00,2012-09-10,107,3001.0,5001.0
7,70010.0,1983.43,2012-10-10,108,3004.0,NaN
8,NaN,NaN,NaN,109,NaN,NaN
9,70003.0,2480.40,2012-10-10,110,3003.0,5003.0


- Detect missing values: display ``True`` or ``False``

In [51]:
# write your solution here



- Identify columns that have at least one missing value

In [52]:
# write your solution here



- Count the number of missing values in each column 

In [53]:
# write your solution here



- Calculate the total number of missing values

In [54]:
# write your solution here



- Count the number of missing values in the first column (answer: 5)

In [55]:
# write your solution here



- Find indices of rows with missing values. Hint: use ``df.index``

In [56]:
# write your solution here



- Drop columns where at least one element is missing

In [57]:
# write your solution here



- Drop rows where at least one element is missing

In [58]:
# write your solution here



- Drop rows having at least two NaNs

In [59]:
# write your solution here



- Drop rows that have NaN's in ``entry_id``, ``customer_id`` or ``salesman_id``. Consult [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) for the required key.

In [60]:
# write your solution here



- Replace NaNs in the ``purch_amt`` column with with the mean of that column. Use extra paramater ``inplace=True`` to modify the DataFrame itself. Consult [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) for more information.

In [61]:
# write your solution here



- Replace NaNs with the values from the previous rows

In [62]:
# write your solution here



- Drop rows having NaN's inplace. Consult [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) for the required key.

In [63]:
# write your solution here



---

**Exercise 2.4.2** Consider the DataFrame defined below. 

In [64]:
df = pd.DataFrame({
'ord_no':[70001,np.nan,70002,70004,np.nan,70005,"--",70010,70003,70012,np.nan,70013],
'purch_amt':[150.5,270.65,65.26,110.5,948.5,2400.6,5760,"?",12.43,2480.4,250.45, 3045.6],
'ord_date': ['?','2012-09-10',np.nan,'2012-08-17','2012-09-10','2012-07-27','2012-09-10','2012-10-10','2012-10-10','2012-06-27','2012-08-17','2012-04-25'],
'entry_id':[101,102,103,104,"?","?",107,108,109,110,111,"?"],
'customer_id':[3002,3001,3001,3003,3002,3001,3001,3004,"--",3002,3001,3001],
'salesman_id':[5002,5003,"?",5001,np.nan,5002,5001,"?",5003,5002,5003,"--"]})
df

,ord_no,purch_amt,ord_date,entry_id,customer_id,salesman_id
0,70001,150.5,?,101,3002,5002
1,NaN,270.65,2012-09-10,102,3001,5003
2,70002,65.26,NaN,103,3001,?
3,70004,110.5,2012-08-17,104,3003,5001
4,NaN,948.5,2012-09-10,?,3002,NaN
5,70005,2400.6,2012-07-27,?,3001,5002
6,--,5760,2012-09-10,107,3001,5001
7,70010,?,2012-10-10,108,3004,?
8,70003,12.43,2012-10-10,109,--,5003
9,70012,2480.4,2012-06-27,110,3002,5002


- Replace the missing values ``?`` and ``--`` with suitable NaNs. Consult [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html?highlight=replace#pandas.DataFrame.replace) for help. Note that some columns contain integer type data. They should remain integer after the modifications are made.

In [65]:
# write your solution here



- Create a heatmap to visualise information about distribution of missing values. (Google for help!)

In [66]:
# write your solution here



---

<!--NAVIGATION-->
< [2.3 – Data Operations in Pandas](L23_Operations_in_Pandas.ipynb) | [Contents](../index.ipynb) | [2.5 – Hierarchical Indexing](L25_Hierarchical_Indexing.ipynb) >


*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; also available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*